# Task 15.4 Solution: Model Registry & Version Control

This notebook provides solutions for the model registry exercise.

---

## Exercise Solution: Complete Model Lifecycle Workflow

**Task:** Create a complete model lifecycle workflow.

Requirements:
1. Create 3 versions of a model (different architectures)
2. Register all versions in MLflow
3. Promote the best one to Production
4. Add descriptions and tags
5. Write a function to load the production model

In [ ]:
import mlflow
import mlflow.pytorch
from mlflow.tracking import MlflowClient
import torch
import torch.nn as nn
import os
import random
from pathlib import Path
from typing import Optional, Dict, Any

# Setup
NOTEBOOK_DIR = Path(os.getcwd())
MLFLOW_DIR = str((NOTEBOOK_DIR / "../mlflow").resolve())
os.makedirs(MLFLOW_DIR, exist_ok=True)

mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
client = MlflowClient()

print(f"MLflow URI: {mlflow.get_tracking_uri()}")

In [ ]:
# Step 1: Define 3 model architectures with different complexities

class TextClassifierSmall(nn.Module):
    """Small model - single hidden layer."""
    def __init__(self, vocab_size: int = 10000, embedding_dim: int = 64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 3)  # 3-class classification
        self.version = "1.0.0-small"
        self.architecture = "small"
        
    def forward(self, x):
        embedded = self.embedding(x)
        pooled = embedded.mean(dim=1)  # Average pooling
        return self.fc(pooled)

class TextClassifierMedium(nn.Module):
    """Medium model - LSTM with attention."""
    def __init__(self, vocab_size: int = 10000, embedding_dim: int = 128, hidden_dim: int = 256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 3)
        self.version = "2.0.0-medium"
        self.architecture = "medium"
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        return self.fc(context)

class TextClassifierLarge(nn.Module):
    """Large model - Bidirectional LSTM with multi-head attention."""
    def __init__(self, vocab_size: int = 10000, embedding_dim: int = 256, hidden_dim: int = 512, num_heads: int = 4):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.MultiheadAttention(hidden_dim * 2, num_heads, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.version = "3.0.0-large"
        self.architecture = "large"
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        pooled = attn_out.mean(dim=1)
        x = self.relu(self.fc1(self.dropout(pooled)))
        return self.fc2(x)

# Create models
models = [
    ("small", TextClassifierSmall()),
    ("medium", TextClassifierMedium()),
    ("large", TextClassifierLarge())
]

print("📋 Model Architectures:")
for name, model in models:
    params = sum(p.numel() for p in model.parameters())
    print(f"   {name}: {params:,} parameters")

In [ ]:
# Step 2: Register all versions

EXPERIMENT_NAME = "TextClassifier-Comparison"
MODEL_NAME = "TextClassifier"

mlflow.set_experiment(EXPERIMENT_NAME)

# Simulated metrics for each architecture
simulated_metrics = {
    "small": {"accuracy": 0.82, "f1": 0.80, "latency_ms": 5, "memory_mb": 50},
    "medium": {"accuracy": 0.88, "f1": 0.87, "latency_ms": 15, "memory_mb": 200},
    "large": {"accuracy": 0.91, "f1": 0.90, "latency_ms": 45, "memory_mb": 800}
}

registered_versions = []

for name, model in models:
    with mlflow.start_run(run_name=f"textclassifier-{name}") as run:
        # Log parameters
        mlflow.log_params({
            "architecture": name,
            "version": model.version,
            "parameters": sum(p.numel() for p in model.parameters())
        })
        
        # Log metrics
        metrics = simulated_metrics[name]
        mlflow.log_metrics(metrics)
        
        # Log model
        sample_input = torch.randint(0, 10000, (1, 50))
        mlflow.pytorch.log_model(
            model,
            artifact_path="model",
            registered_model_name=MODEL_NAME,
            input_example=sample_input.numpy()
        )
        
        print(f"✅ Registered {name} model (acc: {metrics['accuracy']:.2%})")
        registered_versions.append((name, run.info.run_id, metrics))

print(f"\n✅ Registered {len(registered_versions)} model versions!")

In [ ]:
# Step 3: Compare and promote best to Production

# Find all versions
versions = client.search_model_versions(f"name='{MODEL_NAME}'")

print(f"\n📊 Model Version Comparison:")
print("=" * 70)
print(f"{'Version':<10} {'Architecture':<12} {'Accuracy':<12} {'Latency':<12} {'Stage':<12}")
print("-" * 70)

best_version = None
best_accuracy = 0

for v in versions:
    run = mlflow.get_run(v.run_id)
    arch = run.data.params.get('architecture', 'unknown')
    acc = run.data.metrics.get('accuracy', 0)
    latency = run.data.metrics.get('latency_ms', 0)
    
    print(f"{v.version:<10} {arch:<12} {acc:<12.2%} {latency:<12.0f}ms {v.current_stage:<12}")
    
    if acc > best_accuracy:
        best_accuracy = acc
        best_version = v.version

print(f"\n🏆 Best version: {best_version} (accuracy: {best_accuracy:.2%})")

In [ ]:
# Promote best version through stages

print(f"\n🚀 Promoting version {best_version} to Production...")

# First to Staging (testing)
client.transition_model_version_stage(
    name=MODEL_NAME,
    version=best_version,
    stage="Staging"
)
print(f"   ✓ Moved to Staging")

# Then to Production (after "testing")
client.transition_model_version_stage(
    name=MODEL_NAME,
    version=best_version,
    stage="Production",
    archive_existing_versions=True
)
print(f"   ✓ Promoted to Production")

In [ ]:
# Step 4: Add descriptions and tags

# Update registered model description
client.update_registered_model(
    name=MODEL_NAME,
    description="""Text Classification Model

Classifies text into 3 categories.
Supports multiple architecture variants: small, medium, large.

Input: Tokenized text (max 50 tokens)
Output: 3-class probability distribution

Owner: ML Team
Contact: ml-team@company.com
"""
)
print("✅ Updated model description")

# Add version description for production version
client.update_model_version(
    name=MODEL_NAME,
    version=best_version,
    description=f"""Production Model - Large Architecture

Architecture: Bidirectional LSTM with Multi-head Attention
Performance:
- Accuracy: {best_accuracy:.2%}
- F1 Score: 90%
- Latency: ~45ms

Approved for production deployment.
"""
)
print("✅ Updated version description")

# Add tags
tags_to_add = {
    "approved_by": "ml-lead",
    "approval_date": "2024-01-15",
    "deployment_type": "api",
    "min_latency_requirement": "100ms",
    "data_version": "v2.0"
}

for key, value in tags_to_add.items():
    client.set_model_version_tag(MODEL_NAME, best_version, key, value)

print("✅ Added tags to production version")

In [ ]:
# Step 5: Write a function to load the production model

def load_production_model(model_name: str) -> Optional[nn.Module]:
    """
    Load the current production model from MLflow Model Registry.
    
    Args:
        model_name: Name of the registered model
    
    Returns:
        The production model, or None if not found
    """
    try:
        model_uri = f"models:/{model_name}/Production"
        model = mlflow.pytorch.load_model(model_uri)
        print(f"✅ Loaded production model: {model_name}")
        print(f"   URI: {model_uri}")
        print(f"   Type: {type(model).__name__}")
        if hasattr(model, 'version'):
            print(f"   Version: {model.version}")
        return model
    except Exception as e:
        print(f"❌ Failed to load production model: {e}")
        return None

def load_model_by_stage(model_name: str, stage: str = "Production") -> Optional[nn.Module]:
    """
    Load a model by stage (Production, Staging, Archived).
    
    Args:
        model_name: Name of the registered model
        stage: Stage to load from
    
    Returns:
        The model, or None if not found
    """
    try:
        model_uri = f"models:/{model_name}/{stage}"
        return mlflow.pytorch.load_model(model_uri)
    except Exception as e:
        print(f"❌ No model found at {stage}: {e}")
        return None

def get_model_info(model_name: str) -> Dict[str, Any]:
    """
    Get detailed information about all model versions.
    
    Args:
        model_name: Name of the registered model
    
    Returns:
        Dictionary with model information
    """
    info = {"name": model_name, "versions": []}
    
    try:
        model_info = client.get_registered_model(model_name)
        info["description"] = model_info.description
        
        for v in client.search_model_versions(f"name='{model_name}'"):
            info["versions"].append({
                "version": v.version,
                "stage": v.current_stage,
                "tags": v.tags
            })
    except Exception as e:
        info["error"] = str(e)
    
    return info

In [ ]:
# Test the loader functions

print("\n🔧 Testing Model Loader Functions:")
print("=" * 60)

# Load production model
prod_model = load_production_model(MODEL_NAME)

if prod_model:
    # Test inference
    test_input = torch.randint(0, 10000, (2, 50))
    with torch.no_grad():
        output = prod_model(test_input)
        predictions = torch.softmax(output, dim=1)
    
    print(f"\n📊 Test Inference:")
    print(f"   Input shape: {test_input.shape}")
    print(f"   Output shape: {output.shape}")
    print(f"   Predictions: {predictions[0].tolist()}")

In [ ]:
# View final model info

info = get_model_info(MODEL_NAME)

print(f"\n📋 Final Model Registry State:")
print("=" * 60)
print(f"\nModel: {info['name']}")
print(f"\nDescription:\n{info.get('description', 'N/A')[:200]}...")

print(f"\nVersions:")
for v in info['versions']:
    print(f"   v{v['version']} ({v['stage']})")
    if v['tags']:
        for k, val in list(v['tags'].items())[:2]:
            print(f"      {k}: {val}")

---

## Key Takeaways

1. **Multiple Versions**: Different architectures registered as versions of same model
2. **Stage Transitions**: Proper progression through Staging before Production
3. **Documentation**: Descriptions and tags provide context for future use
4. **Programmatic Access**: Functions to load models make deployment easy

---

## Production Checklist

- [x] Multiple versions registered
- [x] Best model identified through metrics comparison
- [x] Production model promoted through staging
- [x] Descriptions added for discoverability
- [x] Tags added for tracking approvals and requirements
- [x] Load functions tested and working

In [ ]:
print("\n✅ Solution complete!")